1) достать инфо про артиста (популярность, жанры, фоловеры, ссылки на изображения [если есть]) (сделано)
2) отсеять тех, где нет в жанрах хип-хоп/рэп | in process 
3) достать дискографию и посчитать количество треков | in process
4) отсеять исполнителей, у которых меньше чем k треков (прикинуть количество из распределения слов на статье на пудинге)
5) проверить доступность лирикса на Genius для каждого исполнителя, динамически отсеивая тех, для которых не набралось k песен с лириксом

In [1]:
import pickle
import re
from os import listdir

import numpy as np
import pandas as pd

In [2]:
retrieved = []
for dir_i in range(1, 6):
    path_to_dir = f"../scripts/1.1-spotify_artists_info_extraction/extracted_artists_info/pt_{dir_i}"
    files = listdir(path_to_dir)
    for pickle_name in files:
        with open(f"{path_to_dir}/{pickle_name}", "rb") as f:
            pickle_part_ = pickle.load(f)
            retrieved.append(pickle_part_)
path_to_missings = (
    "../scripts/1.1-spotify_artists_info_extraction/extracted_artists_info/missings"
)
for missing_pickle in listdir(path_to_missings):
    with open(f"{path_to_missings}/{missing_pickle}", "rb") as f:
        pickle_part_ = pickle.load(f)
        retrieved.append(pickle_part_)
retrieved_df = pd.concat(retrieved)

### Check if it's rap/hip-hop (sub)genre

In [3]:
def genre_check(genre_str):
    patterns = []
    for word_ in ["rap", "hip", "hop"]:
        for start_sym in [" ", "^", "-"]:
            for end_sym in ["$", " ", "-"]:
                patterns.append("%s%s%s" % (start_sym, word_, end_sym))

    pattern = "(" + "|".join(patterns) + ")"
    any_valid_genre = list(
        map(lambda x: True if re.search(pattern, x) else False, genre_str.split("||"))
    )
    return np.any(any_valid_genre)


retrieved_df["is_valid_genre"] = retrieved_df["genre"].apply(genre_check)

In [4]:
valid_artists = retrieved_df[retrieved_df["is_valid_genre"]]
valid_artists

,genre,artist_name,artist_id,followers,genres,image_url,popularity,is_valid_genre
6501,dmv rap,Xanman Music,3yK414LnH89MjOgx7AWKHC,151,dmv rap,,0,True
6502,dmv rap,Ant Glizzy,4oKxjK10P6UR9rbpEhfaUE,3387,dmv rap,https://i.scdn.co/image/ab6761610000e5eb0f6b67...,12,True
6503,dmv rap,DirtyHunnit B.O.N,4neh1KCXmcCRKCeiILwuFF,1633,dmv rap,https://i.scdn.co/image/ab67616d0000b273bf633b...,5,True
6504,dmv rap,HB the Engineer,53wXpZqCp3qGeK6fQAdCte,1213,dmv rap,https://i.scdn.co/image/ab6761610000e5ebb46889...,12,True
6505,dmv rap,Kelow LaTesha,6oUUlugyY0JajHnw3FYAKb,8480,dmv rap||trap queen,https://i.scdn.co/image/ab6761610000e5ebe9c095...,24,True
...,...,...,...,...,...,...,...,...
3996,pinoy hip hop,Paul N Ballin,0Hd8r7OHPwGn4ao1e0PYAu,6041,pinoy hip hop||tagalog rap,https://i.scdn.co/image/ab6761610000e5ebb0052a...,40,True
3997,pinoy hip hop,Roscoe Umali,5sZkYAbskbCE7Zs6KWoxMf,615,pinoy hip hop,https://i.scdn.co/image/ab67616d0000b273f33349...,17,True
3998,pinoy hip hop,8 Ballin',4B65DA1kjKLdgTJydNNDHB,11131,pinoy hip hop||pinoy trap||tagalog rap,https://i.scdn.co/image/ab6761610000e5eb350f1c...,41,True
3999,pinoy hip hop,Hey Its Je,2Cm26ufVCzZu7XVrLH3P6O,21749,opm||pinoy hip hop||tagalog rap,https://i.scdn.co/image/ab6761610000e5ebf37594...,47,True


### Save df for a future extraction of albums spotify info (done in separate scripts)

In [5]:
valid_artists.to_csv(
    "../data/generated/1.2-spotify_artists_info/artists_info.csv", index=False
)